In [84]:
import numpy as np

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn .compose import ColumnTransformer
from sklearn .impute import SimpleImputer
from sklearn .preprocessing import OneHotEncoder
from sklearn .preprocessing import OrdinalEncoder
from sklearn .preprocessing import MinMaxScaler

from sklearn .feature_selection import SelectKBest,chi2
from sklearn .tree import DecisionTreeClassifier

In [86]:
import seaborn as sns

In [87]:
ti= sns.load_dataset('titanic')

In [88]:
ti.drop(columns=['class','who','adult_male','deck','embark_town','alive','alone'],inplace=True)

In [89]:
ti.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


In [90]:
a_train,a_test,b_train,b_test = train_test_split(ti.drop(columns=['survived']),ti['survived'],test_size=0.2)

In [91]:
a_train

,pclass,sex,age,sibsp,parch,fare,embarked
169,3,male,28.0,0,0,56.4958,S
515,1,male,47.0,0,0,34.0208,S
18,3,female,31.0,1,0,18.0000,S
799,3,female,30.0,1,1,24.1500,S
280,3,male,65.0,0,0,7.7500,Q
...,...,...,...,...,...,...,...
85,3,female,33.0,3,0,15.8500,S
295,1,male,NaN,0,0,27.7208,C
89,3,male,24.0,0,0,8.0500,S
551,2,male,27.0,0,0,26.0000,S


In [107]:
a_test

,pclass,sex,age,sibsp,parch,fare,embarked
666,2,male,25.0,0,0,13.0000,S
141,3,female,22.0,0,0,7.7500,S
793,1,male,NaN,0,0,30.6958,C
337,1,female,41.0,0,0,134.5000,C
367,3,female,NaN,0,0,7.2292,C
...,...,...,...,...,...,...,...
492,1,male,55.0,0,0,30.5000,S
120,2,male,21.0,2,0,73.5000,S
249,2,male,54.0,1,0,26.0000,S
715,3,male,19.0,0,0,7.6500,S


In [108]:
b_train

169    0
515    0
18     0
799    0
280    0
      ..
85     1
295    0
89     0
551    0
861    0
Name: survived, Length: 712, dtype: int64

# imputution transform

In [ ]:
trf1 = ColumnTransformer([
    ('impute_age = ',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

# one hot

In [117]:
trf2 = ColumnTransformer([('ohe_se_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])],remainder='passthrough')

In [118]:
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_se_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 6])])

# sCaling  

In [95]:
trf3 = ColumnTransformer([
    ('sCale',MinMaxScaler(),slice(0,10))
])

# feature sele

In [96]:
trf4 = SelectKBest(score_func=chi2,k=5)

# train model

In [97]:
trf5 = DecisionTreeClassifier()

In [103]:
from sklearn.pipeline import Pipeline ,make_pipeline

# Create pipeline

In [124]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [125]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age = ',
                                                  SimpleImputer(), [2]),
                                                 ('impute_embarke',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_se_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('sCale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x00000223A937C720>)),
                ('trf5', DecisionTreeClassifier())])

# pipeline differen make pipeline

In [ ]:
#pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [126]:
from sklearn import set_config
set_config(display='diagram')

In [127]:
pipe.fit(a_train,b_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age = ',
                                                  SimpleImputer(), [2]),
                                                 ('impute_embarke',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_se_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('sCale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x00000223A937C720>)),
                ('trf5', DecisionTreeClassifier())])

In [128]:
y_pred = pipe.predict(a_test)

In [129]:
y_pred

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [130]:
from sklearn .metrics import accuracy_score
accuracy_score(b_test,y_pred)

0.6536312849162011

# Code here

In [ ]:
pipe.named_steps['trf1'].transformers_[0][1]

SimpleImputer()

# eport

In [142]:
import pickle

In [143]:
pickle.dump(pipe,open('pipe.pk1','wb'))